In [1]:
import os, sys, math, datetime, signal
import pathlib
from pathlib import Path
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot as plt
import PIL
import PIL.Image
import seaborn as sns

import tensorflow as tf
# import tensorflow_datasets as tfds
from tensorflow import keras
# from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D,DepthwiseConv2D, MaxPooling2D, AvgPool2D, GlobalAveragePooling2D, BatchNormalization, Concatenate
# from tensorflow.keras.layers import ReLU
# from tensorflow.keras.models import Model
# from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
 
# Import the necessary MLTK APIs
from mltk.core import view_model, summarize_model, profile_model

# import workbench.config.config
from workbench.config.config import initialize
from workbench.utils.utils import create_filepaths, get_file_size
from workbench.utils.utils import parse_model_name, parse_mltk_model_summary
from workbench.data.data import get_lemon_quality_dataset
from workbench.tflite_profiling import get_peak_memory_df, get_tensor_details_df
from workbench.tensorflow import get_layer_details_df

#from dotenv import load_dotenv
import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint
#import deeplake

%load_ext autoreload



In [2]:
%reload_ext autoreload
%autoreload

In [3]:
models_dir = initialize()

In [4]:
automated = False

global model_name

model_name = "mobilenetv3smallSQ_0.1_96_c3_o2_se4l64.MV1"



In [5]:
base_model_name, alpha, resolution, channels, classes, variation = model_name.split("_")
alpha = float(alpha)
resolution = int(resolution)
classes = int(classes.strip("o"))
channels = int(channels.strip("c"))
PROJECT = "model_DB"

# just needed for INT 8 quantization:
BATCH_SIZE = 32
shuffle_seed = 1

In [6]:
models_path, models_summary_path, models_image_path, models_layer_df_path, models_tf_path, models_tflite_path, models_tflite_opt_path = create_filepaths(model_name)

c:\tiny_mlc\tiny_cnn\models


In [7]:
model = keras.models.load_model(models_tf_path)

In [8]:
# view_model(model, tflite=True, build=True)

# Parse model parameters from MLTK summary

In [9]:
total_params, trainable_params, non_trainable_params, MACs, FLOPs = parse_mltk_model_summary(models_summary_path)        

In [10]:
mltk_model_stats = {}

In [11]:
mltk_model_stats["MACs"] = MACs
mltk_model_stats["FLOPs"] = FLOPs
mltk_model_stats["total_params"] = total_params
mltk_model_stats["trainable_params"] = trainable_params
mltk_model_stats["non_trainable_params"] = non_trainable_params

# Creating tflite Models

In [12]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
#converter = tf.lite.TFLiteConverter.from_saved_model(models_path)
tflite_model = converter.convert()


INFO:tensorflow:Assets written to: C:\Users\Susanne\AppData\Local\Temp\tmps9qaapus\assets


INFO:tensorflow:Assets written to: C:\Users\Susanne\AppData\Local\Temp\tmps9qaapus\assets


In [13]:
# Save the model.Y
with open(models_tflite_path, "wb") as f:
    f.write(tflite_model)

In [14]:
mltk_model_stats["model_size_kb"] = get_file_size(models_tf_path)
mltk_model_stats["tflite_model_size_kb"] = get_file_size(models_tflite_path)

File size in bytes is 746912
File size in kilobytes is 729.40625
File size in bytes is 287568
File size in kilobytes is 280.828125


# tflite with INT8 quantization

In [15]:
dataset_path = Path.cwd().joinpath("datasets", "lemon_dataset")
dataset_path.exists()

True

In [16]:
train_ds, val_ds, test_ds, labels = get_lemon_quality_dataset(dataset_path, resolution, resolution, BATCH_SIZE, channels)

Color mode: rgb
Preparing training dataset...
Found 2021 files belonging to 3 classes.
Preparing validation dataset...
Found 252 files belonging to 3 classes.
Preparing test dataset...
Found 255 files belonging to 3 classes.
Class names: ['bad_quality', 'empty_background', 'good_quality']
Train: (TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))
Normalize: True


In [17]:
repr_ds = test_ds.unbatch()

def representative_data_gen():
  for i_value, o_value in repr_ds.batch(1).take(48):
    yield [i_value]
    
converter_opt = tf.lite.TFLiteConverter.from_keras_model(model)

# set the optimization flag
converter_opt.optimizations = [tf.lite.Optimize.DEFAULT]
# enforce integer only quantization
converter_opt.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]


# https://github.com/tensorflow/tensorflow/issues/53293: uint is no longer supported!
#converter_opt.inference_input_type = tf.uint8
#converter_opt.inference_output_type = tf.uint8
converter_opt.inference_input_type = tf.int8
converter_opt.inference_output_type = tf.int8

# provide a representative dataset for quantization
converter_opt.representative_dataset = representative_data_gen

tflite_model_opt = converter_opt.convert()

# Save the model.
with open(models_tflite_opt_path, 'wb') as f:
  f.write(tflite_model_opt)
models_tflite_opt_path

INFO:tensorflow:Assets written to: C:\Users\Susanne\AppData\Local\Temp\tmpyd0nim55\assets


INFO:tensorflow:Assets written to: C:\Users\Susanne\AppData\Local\Temp\tmpyd0nim55\assets
d:\Miniconda\envs\tiny_cnn_6\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


WindowsPath('c:/tiny_mlc/tiny_cnn/models/mobilenetv3smallSQ_0.1_96_c3_o2_se4l64.MV1/mobilenetv3smallSQ_0.1_96_c3_o2_se4l64.MV1_INT8.tflite')

In [18]:
mltk_model_stats["tflite_model_INT8_size_kb"] = get_file_size(models_tflite_opt_path)

File size in bytes is 196664
File size in kilobytes is 192.0546875


In [19]:
mltk_model_stats["tflite_model_size_kb"] = get_file_size(models_tflite_path)

File size in bytes is 287568
File size in kilobytes is 280.828125


# Layer Analytics from TensorFlow Model

In [20]:
layer_details_csv_exists = True
try:
    layer_details_df = get_layer_details_df(models_layer_df_path)
    layer_details_df
except:
    layer_details_csv_exists = False

Reading in c:\tiny_mlc\tiny_cnn\models\mobilenetv3smallSQ_0.1_96_c3_o2_se4l64.MV1\mobilenetv3smallSQ_0.1_96_c3_o2_se4l64.MV1_layers.csv


# Getting peak memory infos

In [21]:
models_peak_memory_path = models_dir.joinpath(model_name, f"{model_name}_peak_memory")

In [22]:
#! python i:\tinyml\tflite-tools\tflite_tools.py -i $models_tflite_opt_path --csv $models_peak_memory_path
! python c:\tiny_mlc\tflite-tools\tflite_tools.py -i $models_tflite_opt_path --csv $models_peak_memory_path
#! explorer $models_peak_memory_path

Writing model analysis to c:\tiny_mlc\tiny_cnn\models\mobilenetv3smallSQ_0.1_96_c3_o2_se4l64.MV1\mobilenetv3smallSQ_0.1_96_c3_o2_se4l64.MV1_peak_memory in CSV format


In [23]:
model_peak_memory_schedule = models_dir.joinpath(model_name, f"{model_name}_peak_memory", "execution_schedule_info.csv")
peak_memory_tensor_details = models_dir.joinpath(model_name, f"{model_name}_peak_memory", "tensor_details.csv")

In [24]:
peak_memory_df = get_peak_memory_df(model_peak_memory_schedule)
peak_memory_df.head(10)

Reading in c:\tiny_mlc\tiny_cnn\models\mobilenetv3smallSQ_0.1_96_c3_o2_se4l64.MV1\mobilenetv3smallSQ_0.1_96_c3_o2_se4l64.MV1_peak_memory\execution_schedule_info.csv
Cleaning up the dataframe.


,index,layer_name,tensor_IDs,RAM_b,operator
0,0,block_0_expand_conv2d,"(0, 131)",64512,mobilenetv3smallSQ/block_0_expand_bn/FusedBatc...
1,1,tf.math.multiply_27,"(131, 132)",73728,mobilenetv3smallSQ/tf.math.multiply_27/Mul
2,2,tf.math.add_27,"(131, 132, 133)",110592,mobilenetv3smallSQ/tf.math.add_27/Add
3,3,tf.clip_by_value_27,"(131, 133, 134)",110592,mobilenetv3smallSQ/tf.clip_by_value_27/clip_by...
4,4,tf.clip_by_value_27,"(131, 134, 135)",110592,mobilenetv3smallSQ/tf.clip_by_value_27/clip_by...
5,5,multiply_27,"(131, 135, 136)",110592,mobilenetv3smallSQ/multiply_27/mul
6,6,block_1_expand_conv2d,"(136, 137)",73728,mobilenetv3smallSQ/tf.nn.relu6_6/Relu6;mobilen...
7,7,batch_normalization_366,"(137, 138)",73728,mobilenetv3smallSQ/tf.nn.relu6_7/Relu6;mobilen...
8,8,conv2d_95,"(138, 139)",55296,mobilenetv3smallSQ/batch_normalization_346/Fus...
9,9,block_2_expand_conv2d,"(139, 140)",110592,mobilenetv3smallSQ/tf.nn.relu6_8/Relu6;mobilen...


In [25]:
peak_memory = peak_memory_df["RAM_b"].max()
peak_memory

113320

In [26]:
mltk_model_stats["peak_memory_b"] = peak_memory

In [27]:
peak_memory_df["RAM_b"].argmax()

10

In [28]:
tensor_df = get_tensor_details_df(peak_memory_tensor_details)
tensor_df

Reading in c:\tiny_mlc\tiny_cnn\models\mobilenetv3smallSQ_0.1_96_c3_o2_se4l64.MV1\mobilenetv3smallSQ_0.1_96_c3_o2_se4l64.MV1_peak_memory\tensor_details.csv
Cleaning up the dataframe.


,index,id,layer_name,shape,size_b,name_long
0,0,0,serving_default_input_2:0,"(1, 96, 96, 3)",27648,serving_default_input_2:0
1,1,131,mobilenetv3smallSQ/block_0_expand_conv2d/Conv2D,"(1, 48, 48, 16)",36864,mobilenetv3smallSQ/block_0_expand_bn/FusedBatc...
2,2,132,mobilenetv3smallSQ/tf.math.multiply_27/Mul,"(1, 48, 48, 16)",36864,tf.math.multiply_27
3,3,133,mobilenetv3smallSQ/tf.math.add_27/Add,"(1, 48, 48, 16)",36864,tf.math.add_27
4,4,134,mobilenetv3smallSQ/tf.clip_by_value_27/clip_by...,"(1, 48, 48, 16)",36864,tf.clip_by_value_27
...,...,...,...,...,...,...
366,366,496,mobilenetv3smallSQ/multiply_53/mul,"(1, 3, 3, 576)",5184,multiply_53
367,367,497,mobilenetv3smallSQ/average_pooling2d_1/AvgPool,"(1, 1, 1, 576)",576,average_pooling2d_1
368,368,498,mobilenetv3smallSQ/flatten_1/Reshape,"(1, 576)",576,flatten_1
369,369,499,mobilenetv3smallSQ/dense_37/BiasAdd,"(1, 2)",2,mobilenetv3smallSQ/dense_37/MatMul;mobilenetv3...


# Logging to Weights & Biases

In [29]:
api = wandb.Api()

# Generate run ids
run_id = wandb.sdk.lib.runid.generate_id()

wandb.init(
    project=PROJECT,
    entity="susbrock",
    name = model_name,
    id=run_id,
    #resume=True
    )

config = wandb.config
config.id = run_id
config.update(mltk_model_stats)

# try:
#     layers_df_wandb = wandb.Table(dataframe=layers_df)
#     wandb.log({"tflite_layers_mltk" : layers_df_wandb})
#     layers_INT8_df_wandb = wandb.Table(dataframe=layers_INT8_df)
#     wandb.log({"tflite_INt8_layers_mltk" : layers_INT8_df_wandb})
# except:
#     print("No analysis from mltk!")

peak_memory_df_wandb = wandb.Table(dataframe=peak_memory_df)
wandb.log({"peak_memory" : peak_memory_df_wandb})

tensor_df_wandb = wandb.Table(dataframe=tensor_df)
wandb.log({"tensor_info" : tensor_df_wandb})

if layer_details_csv_exists:
    layer_details_df_wandb = wandb.Table(dataframe=layer_details_df)
    wandb.log({"tensorflow_layer_details" : layer_details_df_wandb})
else:
    pass

try:
    wandb.log({"tflite_INt8_layers_plt" : wandb.plot.bar(layers_INT8_df_wandb, "index", "macs", title=model_name)})
except:
    pass
wandb.finish()



ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: susbrock. Use `wandb login --relogin` to force relogin


# Profiling tflite models with MLTK profiler

In [30]:
mltk_model_stats["architecture"] = model_name
mltk_model_stats["alpha"] = alpha
mltk_model_stats["img_res"] = resolution
mltk_model_stats["classes"] = classes
mltk_model_stats["channels"] = channels
mltk_model_stats["variation"] = variation

In [32]:
# MLTK profile model reads the mode from a path!

profiling = True

try:
    profiling_results = profile_model(str(models_tflite_path), accelerator=None, build=None)
except:
    profiling= False
    with open("error_list.txt", "a") as f:
        f.write(f"{model_name}, mltk_profiling_error\n")

Profiling model in simulator ...
Using Tensorflow-Lite Micro version: b13b48c (2022-06-08)
Searching for optimal runtime memory size ...
Failed to allocate buffer for model (likely heap memory overflow)


In [33]:
if automated == False:
    mltk_model_stats["opt_RAM_mltk"] = int(input("Please copy the optimal runtime memory size from above: \n"))

In [34]:
if profiling:
    profiling_results.get_summary(exclude_null=False, full_summary=True)
    model_profile = profiling_results.to_dict()


In [35]:
if profiling:
    mltk_model_stats["flash_model_size_b_mltk"] = model_profile["summary"]["tflite_size"]
    mltk_model_stats["RAM_runtime_memory_size_b_mltk"] = model_profile["summary"]["runtime_memory_size"]
    mltk_model_stats["OPS_mltk"] = model_profile["summary"]["ops"]
    mltk_model_stats["macs_mltk"] = model_profile["summary"]["macs"]
    mltk_model_stats["n_unsupported_layers_mltk"] = model_profile["summary"]["n_unsupported_layers"]
    mltk_model_stats["energy_mltk"] = model_profile["summary"]["energy"]

    layers_df = pd.DataFrame.from_dict(model_profile["layers"])
    layers_df["name"] = layers_df["index"].map(str) + ": " + layers_df["opcode"]
    layers_df.insert(2,"name" , layers_df.pop("name"))
    layers_df

    layers_df_path = pathlib.Path.joinpath(models_dir, "layer_df_mltk.pkl")
    layers_df.to_pickle(layers_df_path)

In [36]:
mltk_model_stats

{'MACs': '4.819 M',
 'FLOPs': '9.791 M',
 'total_params': 56874,
 'trainable_params': 53450,
 'non_trainable_params': 3424,
 'model_size_kb': 729.40625,
 'tflite_model_size_kb': 280.828125,
 'tflite_model_INT8_size_kb': 192.0546875,
 'peak_memory_b': 113320,
 'architecture': 'mobilenetv3smallSQ_0.1_96_c3_o2_se4l64.MV1',
 'alpha': 0.1,
 'img_res': 96,
 'classes': 2,
 'channels': 3,
 'variation': 'se4l64.MV1',
 'opt_RAM_mltk': 0}

In [39]:
# code for interrupting indefinetly hanging functions - does not work on Windows!

def handler(signum, frame):
    signame = signal.Signals(signum).name
    print(f'Signal handler called with signal {signame} ({signum})')
    raise OSError("Couldn't open device!")


In [41]:
profiling_INT8 = True


In [ ]:

# Set the signal handler and a 200-second alarm # TODO: this does not work on Windows!
#signal.signal(signal.SIGALRM, handler)

#signal.alarm(10)

# This open() may hang indefinitely
try:
    # %%capture optimal_runtime_INT
    profiling_results_INT8 = profile_model(str(models_tflite_opt_path), accelerator=None, build=False)
except:
    profiling_results_INT8 = False

#signal.alarm(0)    

In [46]:
#profiling_INT8 = False



In [47]:
if profiling_INT8:
    if automated == False:    
        mltk_model_stats["opt_RAM_INT8_mltk"] = int(input("Please copy the optimal runtime memory size from above: \n"))

In [48]:
# from contextlib import redirect_stdout

# with open("runtime.txt", "w", encoding='utf-8') as f:
#     with redirect_stdout(f):
#         print(profile_model(str(models_tflite_opt_path), accelerator=None, build=False))

In [49]:
if profiling_INT8:
    profiling_results_INT8.get_summary()
    model_profile_INT8 = profiling_results_INT8.to_dict()

    mltk_model_stats["flash_model_size_b__INT8_mltk"] = model_profile_INT8["summary"]["tflite_size"]
    mltk_model_stats["RAM_runtime_memory_size_b_INT8_mltk"] = model_profile_INT8["summary"]["runtime_memory_size"]
    mltk_model_stats["OPS_INT8_mltk"] = model_profile_INT8["summary"]["ops"]
    mltk_model_stats["macs_INT8_mltk"] = model_profile_INT8["summary"]["macs"]
    mltk_model_stats["n_unsupported_layers_INT8_mltk"] = model_profile_INT8["summary"]["n_unsupported_layers"]
    mltk_model_stats["energy_INT8_mltk"] = model_profile_INT8["summary"]["energy"]

In [50]:
mltk_model_stats

{'MACs': '4.819 M',
 'FLOPs': '9.791 M',
 'total_params': 56874,
 'trainable_params': 53450,
 'non_trainable_params': 3424,
 'model_size_kb': 729.40625,
 'tflite_model_size_kb': 280.828125,
 'tflite_model_INT8_size_kb': 192.0546875,
 'peak_memory_b': 113320,
 'architecture': 'mobilenetv3smallSQ_0.1_96_c3_o2_se4l64.MV1',
 'alpha': 0.1,
 'img_res': 96,
 'classes': 2,
 'channels': 3,
 'variation': 'se4l64.MV1',
 'opt_RAM_mltk': 0,
 'opt_RAM_INT8_mltk': 0}

In [51]:
if profiling_INT8:
    layers_INT8_df = pd.DataFrame.from_dict(model_profile_INT8["layers"])
    layers_INT8_df["name"] = layers_INT8_df["index"].map(str) + ": " + layers_INT8_df["opcode"]
    layers_INT8_df.insert(2,"name" , layers_INT8_df.pop("name"))
    layers_INT8_df

    layers_df_INT8_path = pathlib.Path.joinpath(models_dir, "layer_df_INT8_mltk.pkl")
    layers_INT8_df.to_pickle(layers_df_INT8_path)

# Logging to Weights & Biases - part 2

In [52]:
wandb.init(
    project=PROJECT,
    entity="susbrock",
    name = model_name,
    id=run_id,
    resume="allow"
    )

# config = wandb.config
# config.id = run_id
config.update(mltk_model_stats)

try:
    layers_df_wandb = wandb.Table(dataframe=layers_df)
    wandb.log({"tflite_layers_mltk" : layers_df_wandb})
    layers_INT8_df_wandb = wandb.Table(dataframe=layers_INT8_df)
    wandb.log({"tflite_INt8_layers_mltk" : layers_INT8_df_wandb})
except:
    print("No analysis from mltk!")

# peak_memory_df_wandb = wandb.Table(dataframe=peak_memory_df)
# wandb.log({"peak_memory" : peak_memory_df_wandb})

# tensor_df_wandb = wandb.Table(dataframe=tensor_df)
# wandb.log({"tensor_info" : tensor_df_wandb})

# if layer_details_csv_exists:
#     layer_details_df_wandb = wandb.Table(dataframe=layer_details_df)
#     wandb.log({"tensorflow_layer_details" : layer_details_df_wandb})
# else:
#     pass

# try:
#     wandb.log({"tflite_INt8_layers_plt" : wandb.plot.bar(layers_INT8_df_wandb, "index", "macs", title=model_name)})
# except:
#     pass
wandb.finish()


No analysis from mltk!
